<a href="https://colab.research.google.com/github/supertime1/BP_PPG/blob/master/Data%20Processing/Biometric_Info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is to generate biometric information of patients in wfdb matched dataset

In [1]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard
import numpy as np
import os
import shutil
import glob
import wfdb
from wfdb import processing
from datetime import datetime
import sklearn.metrics
import itertools
import io
import pickle

In [2]:
directory = "C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/PATIENTS.csv"
bio =pd.read_csv(directory,encoding= 'unicode_escape')

In [26]:
date_ls = []
id_ls = []
for name in glob.glob("D:/WFDB//matched/p*/*/p*[!n].hea"):
  idx = name.index('-')
  r_date = name[idx+1:idx+11]
  s_id = name.split('\\')[2]
  date_ls.append(r_date)
  id_ls.append(s_id)

d = {'Subject ID': id_ls, 'Receive Date': date_ls}
matched_patient = pd.DataFrame(data = d)
print("There are total ", len(date_ls), "header files")

There are total  22317 header files


In [28]:
matched_patient.to_csv("C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/PATIENT_MATCHED.csv")

In [14]:
idx = hd_names[0].index('-')
r_date = hd_names[0][idx+1:idx+11]
s_id = hd_names[0].split('\\')[2]

In [24]:
d = {'Subject ID': [s_id], 'Receive Date': [r_date]}

In [25]:
pd.DataFrame(data = d)

,Subject ID,Receive Date
0,p000020,2183-04-28


In [ ]:
qualified_names = [] #a list of file names that contain both PPG and ABP waveforms
for name in hd_names:
  record = wfdb.rdheader(name)
  if 'ABP' in record.sig_name and 'PLETH' in record.sig_name and record.sig_len >= 75000: #only extract records contrains ECG lead I and PPG, and >=10min
    qualified_names.append(name)
print('There are total', len(qualified_names), 'records that have both ABP and PPG with more than 10min recording')

with open('D:/WFDB//matched/BP/qualified_names.pkl', "wb") as fp:
  pickle.dump(qualified_names, fp)